Скачайте корпус текстов, обработайте его с помощью UDPipe, извлеките все группы "глагол + прямое дополнение, выраженное существительным" (не учитывайте глаголы, которые встречаются в корпусе менее 50 раз).

In [0]:
!ls


### **этот модуль делаем на компе, загружаем готовый файл** ".conllu"

In [0]:
#'этот модуль делаем на комппе, после чего загружаем готовый файл testset2.conllu в colab
!C:\Users\Administrator\NLP_Admin\udpipe-1.2.0-bin\bin-osx\udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
D:\__NLP\Model_UDPipe\russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu

In [1]:

with open('testset2.conllu', 'r', encoding='utf-8') as f: #, encoding='utf-8'
    text = f.read()
print(len(text))

12434505


In [0]:
from nltk.parse import DependencyGraph


In [0]:
trees = []

with open('testset2.conllu', 'r', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')
    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [0]:
g = DependencyGraph(trees[8], top_relation_label='root')

In [0]:
_FILTER_RELS = ['obj']
def get_subtree(nodes, node):
    res=[]
    if 'deps' in nodes[node]:
        for rel in nodes[node]['deps']:
            if rel in _FILTER_RELS:
                for dep in nodes[node]['deps'][rel]:
                    res.append(dep)
    return res          
                          

In [0]:
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

In [0]:
result = []
verbs_count={}
verbs=[]
N=0
for t in trees:
  try:
    g = DependencyGraph(t, top_relation_label='root')
    np_list = []
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'VERB':
          verb=g.nodes[n]['word'].lower()
          verbs.append(verb)
          
          np = list(sorted(get_subtree(g.nodes, n)))
          for i in np:
            w2=g.nodes[n]['word']+' '+g.nodes[i]['word']
            np_list.append((g.nodes[n]['word'], g.nodes[i]['word']))
    
    if len(np_list)>0:
      for el in np_list:
        result.append(el)
  except:
    print('ERROR:', t)

#verbs=set(verbs)
#print(verbs_count)
#print(verbs)
result

In [0]:
def select_pair(pair, List_verbs):
  #print(len(pair),len(List_verbs))
  res=[]
  for p in pair:
    for x in p:
      word_pair=set(x.split())
      #print(word_pair)
      #print(word_pair.intersection(List_verbs))

      if len(word_pair.intersection(List_verbs))>0:
        res.append(p)
     
  return res


In [0]:
list_pair=select_pair(result, verbs)
list_pair[10:20]

In [0]:
from collections import Counter
verbs_count = Counter(verbs)
#print(verbs_count)
verbs_more_50 = [ w for w in verbs_count.elements() if verbs_count[w] >= 50]

list_pair=select_pair(result, verbs_more_50)

verbs_more_50 =set(verbs_more_50)
#verbs_50 =Counter(verbs_more_50)
list_pair
#verbs_more_50

#print(list_pair)


In [28]:
#преобразуем пару глагол + сущ в первонач форму
verb_noun_norm=[]
#print(List_pair)
for p in list_pair:
  if len(p[0])>1 and len(p[1])>1: #исключем случайный мусор, который попал
    el1=m.parse(p[0])[0].normal_form
    el2=m.parse(p[1])[0].normal_form
    norm_pair=(el1,el2)
    verb_noun_norm.append(norm_pair)

len(verb_noun_norm)

1041

In [29]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('russian')
#stop

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**2. Оцените полученные словосочетания следующими метриками: log-likelihood, dice, PMI (можно использовать nltk.collocations).**

In [0]:
!pip install pymorphy2


In [0]:
#import nltk
import pymorphy2
#from nltk import collocations

from pymorphy2 import MorphAnalyzer

m = MorphAnalyzer()

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [0]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(t)[0].normal_form for t in tokens]
    return lemmas

In [0]:
import nltk
from nltk import collocations 

In [37]:
with open('testset2.txt', 'r', encoding='utf-8') as f: #, encoding='utf-8'
    testset2 = f.read()
len(testset2)

1037281

In [0]:
finder_test = collocations.BigramCollocationFinder.from_documents([normalize(testset2)])

In [0]:
finder_test.apply_ngram_filter(lambda w1, w2: (w1, w2) not in verb_noun_norm)

**-log-likeLlihood**

In [0]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().likelihood_ratio)

In [0]:
likelihood_100 = finder_test.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 100)
#len(likelihood_100)

**-Dice**

In [0]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().dice)

In [0]:
dice_100 = finder_test.nbest(collocations.BigramAssocMeasures().dice, 100)
dice_100

**-PMI**

In [0]:
finder_test.score_ngrams(collocations.BigramAssocMeasures().pmi)

In [0]:
pmi_100 = finder_test.nbest(collocations.BigramAssocMeasures().pmi, 100)
pmi_100

**3. Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: возьмите словосочетания, которые попадают в топ-100 по всем метрикам, пересеките со словарем глагольной сочетаемости.**

In [0]:
GS_coll=set(likelihood_100).intersection(dice_100)
GS_coll=set(GS_coll).intersection(pmi_100)
GS_coll

In [0]:
import re

with open('verb_coll.txt', 'r', encoding='utf-8') as f:
    verb_txt = f.read()

# из корпуса вытаскиваем пары verb-noun
verb_coll=[]
pattern='VN.*\n'   
match = re.findall(pattern, verb_txt) 
for s in match:
    s=s.replace('VN','')
    s = re.sub(r'\s+', ' ', s, flags=re.UNICODE)
    s=s.replace('\n','')
    
    words = s.split()
    if len(words)==2:
        coll_lemm = (m.parse(words[0])[0].normal_form, m.parse(words[1])[0].normal_form)
        verb_coll.append(coll_lemm)
#print(verb_coll)

In [57]:
GS = set(GS_coll).intersection(set(verb_coll))
GS

{('вынести', 'определение'),
 ('вынести', 'постановление'),
 ('вынести', 'приговор'),
 ('вынести', 'решение'),
 ('подать', 'иск'),
 ('принять', 'решение'),
 ('принять', 'участие'),
 ('удовлетворить', 'иск'),
 ('удовлетворить', 'требование')}

**4. Добавьте в ЗС словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть), объясните свой выбор.**

In [0]:
add_coll={
 ('вынести', 'вердикт'),
 ('выплатить', 'дивиденд'),
 ('выплатить', 'компенсация'),
 ('выплатить', 'штраф'),
 ('отклонить', 'апелляция'),
 ('отклонить', 'жалоба'),
 ('отклонить', 'иск'),
 ('отклонить', 'ходатайство'),
 ('подать', 'апелляция'),
 ('подать', 'иск'),
 ('подать', 'кассация'),
 ('подтвердить', 'законность'),
 ('подтвердить', 'запрет'),
 ('подтвердить', 'информация'),
 ('подтвердить', 'правомерность'),
 ('подтвердить', 'правомочность'),
 ('признать', 'правота'),
 ('принять', 'решение'),
 ('принять', 'участие'),
 ('удовлетворить', 'жалоба'),
 ('удовлетворить', 'ходатайство')}
 

In [59]:
GS=GS.union(add_coll)
len(GS) #размер множества для расчета корреляций
#GS

27

**5. Оцените ранговую корреляцию (коэффициент Спирмена) результатов по каждой метрике с ЗС. Можно использовать scipy.stats.spearmanr. Опишите ошибки каждой метрики.**

In [0]:
from scipy.stats import spearmanr

In [0]:
like_27 = finder_test.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 27)
dice_27 = finder_test.nbest(collocations.BigramAssocMeasures().dice, 27)
pmi_27 = finder_test.nbest(collocations.BigramAssocMeasures().pmi, 27)


In [0]:
finder_test.apply_ngram_filter(lambda w1, w2: (w1, w2) not in GS)

In [0]:
def transform_to_row(list):
    row = []
    for el in list:
        row.append(' '.join(el))
    return row

In [0]:
row=transform_to_row(like_27)
row

**likelihood spearmanr**

In [0]:
like_27_GS = finder_test.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 27)
rho, p = spearmanr(transform_to_row(like_27_GS), transform_to_row(like_27))
print(f'rho_like: {rho}\np_like: {p}')
#like_27_GS

rho_like: 0.34859584859584863
p_like: 0.07474551078913241


In [77]:
dice_27_GS = finder_test.nbest(collocations.BigramAssocMeasures().dice, 27)
rho, p = spearmanr(transform_to_row(dice_27_GS), transform_to_row(dice_27))
print(f'rho_dice: {rho}\np_dice: {p}')


rho_dice: 0.5079365079365078
p_dice: 0.006833748394532331


In [78]:
pmi_27_GS = finder_test.nbest(collocations.BigramAssocMeasures().pmi, 27)
rho, p = spearmanr(transform_to_row(pmi_27_GS), transform_to_row(pmi_27))
print(f'rho_pmi: {rho}\np_pmi: {p}')


rho_pmi: 0.2442002442002442
p_pmi: 0.21961664482521304
